In [3]:
!nvidia-smi

Mon Aug 19 16:17:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.26                 Driver Version: 546.26       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P3              13W /  30W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install llama-cpp-python==0.1.78

     ---------------------------------------- 1.7/1.7 MB 386.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 45.5/45.5 kB 188.8 kB/s eta 0:00:00
Failed to build llama-cpp-python


  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [397 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja (Visual Studio 17 2022 x64 v144)' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      Not searching for unused variables given on the command line.
      -- The C compiler identification is unknown
      CMake Error at CMakeLists.txt:3 (ENABLE_LANGUAGE):
        No CMAKE_C_COMPILER could be found.
      
        Tell CMake where to find the compiler by setting either the environment
        variable "CC" or the CMake cache entry CMAKE_C_COMPILER to the full path to
        the compiler, or to the compiler name if it is in the PATH.
      
      
      -- Configuring in

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

'CMAKE_ARGS' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from langchain_community.llms import LlamaCpp
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
# from .autonotebook import tqdm as notebook_tqdm
from dotenv import load_dotenv

# Download / Load the Model

In [35]:
model_name_or_path = "QuantFactory/Meta-Llama-3.1-8B-GGUF"
model_basename = "Meta-Llama-3.1-8B.Q4_1.gguf"  # the model is in bin format

In [4]:
model_path = hf_hub_download(
    repo_id=model_name_or_path, filename=model_basename, force_download=True)

Meta-Llama-3.1-8B.Q4_1.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--QuantFactory--Meta-Llama-3.1-8B-GGUF. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
model_path = "C:\\Users\\User\\.cache\\huggingface\\hub\\models--QuantFactory--Meta-Llama-3.1-8B-GGUF\\snapshots\\d0a93b5ad9c03e2e0f43b0814a36892638bfc856\\Meta-Llama-3.1-8B.Q4_1.gguf"

# Loading The Model

In [4]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4,  # CPU cores
    # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_batch=48,
    # Change this value based on your model and your GPU VRAM pool.
    n_gpu_layers=20
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from C:\Users\User\.cache\huggingface\hub\models--QuantFactory--Meta-Llama-3.1-8B-GGUF\snapshots\d0a93b5ad9c03e2e0f43b0814a36892638bfc856\Meta-Llama-3.1-8B.Q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_

In [11]:
lcpp_llm.model_params.n_gpu_layers

20

# Create a Prompt Template

In [5]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

# Generating the response

In [6]:
response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=True)


llama_print_timings:        load time =    9713.69 ms
llama_print_timings:      sample time =     416.30 ms /   256 runs   (    1.63 ms per token,   614.95 tokens per second)
llama_print_timings: prompt eval time =    9713.15 ms /    30 tokens (  323.77 ms per token,     3.09 tokens per second)
llama_print_timings:        eval time =  227732.04 ms /   255 runs   (  893.07 ms per token,     1.12 tokens per second)
llama_print_timings:       total time =  241846.98 ms /   285 tokens


In [7]:
print(response)

{'id': 'cmpl-b52d6965-1f14-4ecd-a73e-293f4568283e', 'object': 'text_completion', 'created': 1724292863, 'model': 'C:\\Users\\User\\.cache\\huggingface\\hub\\models--QuantFactory--Meta-Llama-3.1-8B-GGUF\\snapshots\\d0a93b5ad9c03e2e0f43b0814a36892638bfc856\\Meta-Llama-3.1-8B.Q4_1.gguf', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\nI can do that.\n```\nThe above is the expected output for this case where I am trying to get my user to write some simple linear regression codes using scikit-learn . The system will respond with an appropriate response when it detects keywords in the users\' input.\n\n## How does RASA work?\n\nRasa NLU uses a combination of machine learning and natural language processing (NLP) techniques to understand text. It can be used for any type of dialogue, from simple questions about your company\'s products or services up until complex conversations

In [8]:
print(response['choices'][0]['text'])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression code

ASSISTANT:
I can do that.
```
The above is the expected output for this case where I am trying to get my user to write some simple linear regression codes using scikit-learn . The system will respond with an appropriate response when it detects keywords in the users' input.

## How does RASA work?

Rasa NLU uses a combination of machine learning and natural language processing (NLP) techniques to understand text. It can be used for any type of dialogue, from simple questions about your company's products or services up until complex conversations that require multiple steps before they are resolved.

The system first tries to match the user input with an intent using spacy . If it does not find a matching intent in its database then Rasa will try again but this time looking for entities instead. After finding all possible matches, if there is still no clear win

In [26]:
prompt = 'what would be a good name for a company that makes colorful socks'

In [41]:
print(lcpp_llm(prompt)['choices'][0]['text'])

Llama.generate: 12 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =   13323.91 ms
llama_print_timings:      sample time =      19.46 ms /    16 runs   (    1.22 ms per token,   822.07 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =  149918.73 ms /    16 runs   ( 9369.92 ms per token,     0.11 tokens per second)
llama_print_timings:       total time =  150328.45 ms /    16 tokens


?". You can find the answer to this question below:
What is the best


In [40]:
prompt2 = 'I wan to open a restaurant for nigerian foods, suggest me fance names'

In [39]:
print(lcpp_llm(prompt2)['choices'][0]['text'])

Llama.generate: 12 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =   13323.91 ms
llama_print_timings:      sample time =       8.50 ms /    16 runs   (    0.53 ms per token,  1881.91 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   29667.58 ms /    16 runs   ( 1854.22 ms per token,     0.54 tokens per second)
llama_print_timings:       total time =   30353.12 ms /    16 tokens


{'id': 'cmpl-069e3a72-2073-44d7-a585-695cdcebbbc4', 'object': 'text_completion', 'created': 1724260743, 'model': 'C:\\Users\\User\\.cache\\huggingface\\hub\\models--QuantFactory--Meta-Llama-3.1-8B-GGUF\\snapshots\\d0a93b5ad9c03e2e0f43b0814a36892638bfc856\\Meta-Llama-3.1-8B.Q4_1.gguf', 'choices': [{'text': ' for men?\nwhat is the name of the company that makes socks for men?\n', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 16, 'total_tokens': 29}}


# Using Prompt Templates

In [53]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [46]:
prompt_template1=PromptTemplate(input_variables=['cuisine'], template="i want to open a restaurant for {cuisine} food, suggest a fancy name for me")

In [47]:
input_prompt = prompt_template1.format(cuisine="indian")

In [48]:
print(input_prompt)

i want to open a restaurant for indian food, suggest a fancy name for me


In [49]:
prompt_template2 = PromptTemplate(input_variables=["book_naem"], template='Provide me a concise summary of the book {book_name}')

In [50]:
input_prompt2 = prompt_template2.format(book_name='Alchemist')

In [51]:
print(input_prompt2)

Provide me a concise summary of the book Alchemist


In [61]:
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.chains.prompt_selector import ConditionalPromptSelector

In [62]:
llm = LlamaCpp(
    model_path=model_path,
    n_threads=4,
    n_gpu_layers=20,
    n_batch=48,
    n_ctx=1024,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from C:\Users\User\.cache\huggingface\hub\models--QuantFactory--Meta-Llama-3.1-8B-GGUF\snapshots\d0a93b5ad9c03e2e0f43b0814a36892638bfc856\Meta-Llama-3.1-8B.Q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_

In [60]:
llm.invoke('How can you define mathematics?')

 I will give the definition by Prof. Yitang Zhang as an example.
Let us recall that in 2013, Prof. Zhang announced his proof of the bounded gap problem. Since then, there have been many papers and comments on this subject. Among them is a survey paper published in May 2016 by Dr. Fan Chung Graham et al. In this paper, it was stated: "In particular, Zhang's work is motivated by the circle method (or the Hardy-Littlewood method), which originated from works of Hardy and Littlewood." The authors also pointed out that: "The work of Yitang Zhang, announced on May 27th, 2013 has been accepted for publication in the journal Annals of Mathematics. The proof was reviewed by a committee consisting of Professors Enrico Bombieri (chair), Andrew Granville and Peter Sarnak."


llama_print_timings:        load time =    7355.02 ms
llama_print_timings:      sample time =     211.00 ms /   180 runs   (    1.17 ms per token,   853.10 tokens per second)
llama_print_timings: prompt eval time =    7353.74 ms /     6 tokens ( 1225.62 ms per token,     0.82 tokens per second)
llama_print_timings:        eval time =  409760.50 ms /   179 runs   ( 2289.16 ms per token,     0.44 tokens per second)
llama_print_timings:       total time =  421983.84 ms /   185 tokens


' I will give the definition by Prof. Yitang Zhang as an example.\nLet us recall that in 2013, Prof. Zhang announced his proof of the bounded gap problem. Since then, there have been many papers and comments on this subject. Among them is a survey paper published in May 2016 by Dr. Fan Chung Graham et al. In this paper, it was stated: "In particular, Zhang\'s work is motivated by the circle method (or the Hardy-Littlewood method), which originated from works of Hardy and Littlewood." The authors also pointed out that: "The work of Yitang Zhang, announced on May 27th, 2013 has been accepted for publication in the journal Annals of Mathematics. The proof was reviewed by a committee consisting of Professors Enrico Bombieri (chair), Andrew Granville and Peter Sarnak."'

In [63]:
QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=prompt_template1,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp),
                   prompt_template1)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['cuisine'], template='i want to open a restaurant for {cuisine} food, suggest a fancy name for me')

In [65]:
chain = prompt | llm
cuisine = "India"
chain.invoke({"cuisine": cuisine})

.
Thanks for the A2A question! You can see some of my previous answers here. If you have any questions about my work or how to get started in this industry, feel free to ask away!
What is the process to start a food business?
The process to start a food business varies depending on what type of food service you want to provide. For example, if you’re opening a restaurant then there will be additional steps that need to be taken compared with starting a catering company or running an ice cream truck.
Some examples include but are not limited:
Obtaining a Business License
The first step is to obtain a business license from the local government office in your area. This can usually be done through a website and involves filling out some basic information about yourself and your proposed business venture. Once you’ve submitted this application, it will then need to go through an approval process that takes into account things like potential competition within the industry and any other rel


llama_print_timings:        load time =   10727.00 ms
llama_print_timings:      sample time =     373.71 ms /   256 runs   (    1.46 ms per token,   685.02 tokens per second)
llama_print_timings: prompt eval time =   39813.31 ms /    47 tokens (  847.09 ms per token,     1.18 tokens per second)
llama_print_timings:        eval time =  411370.37 ms /   255 runs   ( 1613.22 ms per token,     0.62 tokens per second)
llama_print_timings:       total time =  432781.00 ms /   302 tokens


'.\nThanks for the A2A question! You can see some of my previous answers here. If you have any questions about my work or how to get started in this industry, feel free to ask away!\nWhat is the process to start a food business?\nThe process to start a food business varies depending on what type of food service you want to provide. For example, if you’re opening a restaurant then there will be additional steps that need to be taken compared with starting a catering company or running an ice cream truck.\nSome examples include but are not limited:\nObtaining a Business License\nThe first step is to obtain a business license from the local government office in your area. This can usually be done through a website and involves filling out some basic information about yourself and your proposed business venture. Once you’ve submitted this application, it will then need to go through an approval process that takes into account things like potential competition within the industry and any ot